In [11]:
import numpy as np
import pandas as pd

In [13]:
nums = list(range(1, 19))
meta = [str(i)  if i > 9 else '0'+str(i) for i in nums]
print(meta)
path = '/home/bsb2144/daart_utils/data/huga'
s = '01'
good_sess = []
# load labels
for s in meta:
    #print('session ' + s)
    lab_path = path + '/labels-hand/padd_sessn_' + s + '_labels.csv'
    lab_df = pd.read_csv(lab_path, index_col=0)
    #lab_df.drop('bicycling', axis=1, inplace=True)
    counts = list(lab_df.sum(axis=0))
    if 0 not in counts:
        print('session ' + s)
        print('total:', lab_df.shape)
        print(lab_df.sum(axis=0))
        print('')
        good_sess.append(s)

['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18']
session 01
total: (70787, 9)
background       1600
walking         31296
running          5995
going_up         8651
going_down       8842
sitting          4357
sitting_down     1510
standing_up      1577
standing         6959
dtype: int64

session 02
total: (22337, 9)
background       600
walking         9483
running         1474
going_up        2250
going_down      2579
sitting         1787
sitting_down     728
standing_up      675
standing        2761
dtype: int64

session 03
total: (38670, 9)
background       2400
walking         11281
running          1706
going_up         4880
going_down       5767
sitting          3241
sitting_down      723
standing_up      1352
standing         7320
dtype: int64

session 04
total: (39763, 9)
background       1500
walking         21021
running          2972
going_up         3955
going_down       2955
sitting          3312
sitting_down    

In [15]:
nums = list(range(1, 19))
meta = [str(i)  if i > 9 else '0'+str(i) for i in nums]
print(meta)
path = '/home/bsb2144/daart_utils/data/huga'
s = '01'
good_sess = []
# load labels
for s in meta:
    #print('session ' + s)
    lab_path = path + '/labels-hand/padd_sessn_' + s + '_labels.csv'
    lab_df = pd.read_csv(lab_path, index_col=0)
    #lab_df.drop('bicycling', axis=1, inplace=True)
    counts = list(lab_df.sum(axis=0))
    if 0 not in counts:
        print('session ' + s)
        print('total:', lab_df.shape)
        print(lab_df.sum(axis=0))
        print('')
        good_sess.append(s)

['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18']
session 01
total: (70787, 9)
background       1600
walking         31296
running          5995
going_up         8651
going_down       8842
sitting          4357
sitting_down     1510
standing_up      1577
standing         6959
dtype: int64

session 02
total: (22337, 9)
background       600
walking         9483
running         1474
going_up        2250
going_down      2579
sitting         1787
sitting_down     728
standing_up      675
standing        2761
dtype: int64

session 03
total: (38670, 9)
background       2400
walking         11281
running          1706
going_up         4880
going_down       5767
sitting          3241
sitting_down      723
standing_up      1352
standing         7320
dtype: int64

session 04
total: (39763, 9)
background       1500
walking         21021
running          2972
going_up         3955
going_down       2955
sitting          3312
sitting_down    

In [3]:
train_vids = ['05', '07', '09', '12', '14']
test_vids = ['06', '08', '11', '13', '17']

In [16]:
print(lab_df.sum(axis=0))

background       1600
walking         14980
running          3744
going_up         9335
going_down       7743
sitting          7291
sitting_down     1105
standing_up      1047
standing         9319
dtype: int64


In [6]:
# 500 labels per class across all videos
# try other valuse - 500, 1000, 1500, 2000, 2500
# loop throgh behaviors
#    randomly select participant - ensure the class is present
#    choose random index with behavior
#      - exclude last 50 frames of segment
#    set index: index+50
# make sure not to use same bout twice

# validate

# fit tcn for 5 diff models

# 5 seeds per train frac -diff subset of labeled data

In [7]:
# train - 5 - frac based on number of labels per class, not num of videos
# test - 5

# remove all subjects without elevator and/or walking
# then trim down to 10
# test vids need all behaviors present


# gen all 25 datasets and validate

In [20]:
path = '/home/bsb2144/daart_utils/data/huga'

train_dfs = []
# load labels
for s in train_vids:
    #print('session ' + s)
    lab_path = path + '/labels-hand/padd_sessn_' + s + '_labels.csv'
    lab_df = pd.read_csv(lab_path, index_col=0)
    train_dfs.append(lab_df)

In [21]:
def get_labels_start_end_time(array, bg_class=[0], use=1):
    labels = []
    starts = []
    ends = []
    
    frame_wise_labels = array.copy()
    #print('pre', frame_wise_labels[:100])
    frame_wise_labels[array != use] = 0
    #print('post', frame_wise_labels[:100])
    
    last_label = frame_wise_labels[0]
    if frame_wise_labels[0] not in bg_class:
        labels.append(frame_wise_labels[0])
        starts.append(0)
    for i in range(len(frame_wise_labels)):
        if frame_wise_labels[i] != last_label:
            if frame_wise_labels[i] not in bg_class:
                labels.append(frame_wise_labels[i])
                starts.append(i)
            if last_label not in bg_class:
                ends.append(i)
                
            last_label = frame_wise_labels[i]
            
    if last_label not in bg_class:
        ends.append(i + 1)
        
    ends = [e-50 for e in ends]
    lens = [e-s for e,s in zip(ends, starts)]
    
    to_del = []
    for l, ln in enumerate(lens): 
        if ln < 1:
            to_del.append(l)
            
    offset = 0
    for l in to_del:
        starts.pop(l-offset)
        ends.pop(l-offset)
        labels.pop(l-offset)
        lens.pop(l-offset)
        offset += 1
            
    lens = [e-s for e,s in zip(ends, starts)]
    return (labels, starts, ends, lens)

In [22]:
import random

path = '/home/bsb2144/daart_utils/data/huga' 

# save dfs as np arrays
names = ['background','walking', 'running', 'going_up', 'going_down', 'sitting',
                      'sitting_down', 'standing_up', 'standing']

# main loop
goals = [50, 100, 250, 500, 1000, 2000, 3000]
vs = ['v1', 'v2', 'v3', 'v4', 'v5']
seeds = [1, 1001, 2001, 3001, 4001]

#v = 'v1'
#goal = 500

for goal in goals:
    for v, seed in zip(vs, seeds):

        labels_int = []
        for df in train_dfs:
            temp = np.zeros(df.shape[0])
            #print(temp.shape)
            inds = list(df.idxmax(axis=1))
            np_inds = np.array([names.index(i) for i in inds])
            labels_int.append(np_inds)

        classes = ['walking', 'running', 'going_up', 'going_down', 'sitting',
                              'sitting_down', 'standing_up', 'standing']

        num_labeled = np.zeros(len(classes))

        random.seed(seed)

        # create array to hold new labels
        new_labels = [np.zeros_like(a) for a in labels_int]


        # iterate through classes
        for c, cl in enumerate(classes):
            print('cl', cl)
            tries = 0
            # keep adding labeled chunks until we reach goal
            while num_labeled[c] < goal:
                print('num', num_labeled[c])
                random.seed(seed)

                # choose a video and update seed
                vid = random.randint(0, len(train_vids)-1)
                seed += 1
                arr = labels_int[vid]
                #print(arr[:10], len(arr))

                # choose index  
                labels, starts, ends, lens = get_labels_start_end_time(np.array(arr), bg_class=[0], use=(c+1))


                # create list of all valid start inds
                start_inds = []
                for s,e in zip(starts, ends):
                    start_inds += list(range(s,e))

                # pick a start index
                random.seed(seed)
                #print(goal, cl)
                #print('len start', len(start_inds))
                if len(start_inds) < 1:
                    tries += 1
                    if tries > 5:
                        break
                    continue
                si_ind = random.randint(0, len(start_inds)-1)
                seed += 1
                si = start_inds[si_ind]

                # update new labels array
                #print('pre', new_labels[vid][si:si+51])
                new_labels[vid][si:si+50] = (c+1)
                #print('post', new_labels[vid][si:si+51])

                # remove chose indexes from list of possible inds
                labels_int[vid][si:si+50] = 0

                # update num labeled
                num_labeled[c] += 50
                #print('num labeled class', (c+1), num_labeled[c])

        # create df based on new labels and save
        new_dfs = []
        for arr in new_labels:
            lab_np = np.zeros((len(arr), 11))
            for ind, l in enumerate(arr):
                lab_np[ind, l] = 1
            lab_df = pd.DataFrame(lab_np, dtype=int)
            new_dfs.append(lab_df)

        train_vids = ['05', '07', '09', '12', '14']
        test_vids = ['06', '08', '11', '13', '17']

        # save labels
        # format of file - 'sess_' + s + '_' + num_labels + '_' + seed version + '_labels.csv'
        # i.e. sess_05_500_v1_labels.csv
        for d, df in enumerate(new_dfs):
            lab_path = path + '/labels-hand/' + 'sess_' + train_vids[d] + '_' + str(goal) + '_' + v + '_labels.csv'
            df.columns = ['background', 'walking', 'running', 'going_up', 'going_down', 'sitting',
                             'sitting_down', 'standing_up', 'standing', 'down_elevator', 'up_elevator']
            df.to_csv(lab_path)
        

cl walking
num 0.0
cl running
num 0.0
cl going_up
num 0.0
cl going_down
num 0.0
cl sitting
num 0.0
cl sitting_down
num 0.0
cl standing_up
num 0.0
cl standing
num 0.0
cl walking
num 0.0
cl running
num 0.0
cl going_up
num 0.0
cl going_down
num 0.0
cl sitting
num 0.0
cl sitting_down
num 0.0
cl standing_up
num 0.0
cl standing
num 0.0
cl walking
num 0.0
cl running
num 0.0
cl going_up
num 0.0
cl going_down
num 0.0
cl sitting
num 0.0
cl sitting_down
num 0.0
cl standing_up
num 0.0
cl standing
num 0.0
cl walking
num 0.0
cl running
num 0.0
cl going_up
num 0.0
cl going_down
num 0.0
cl sitting
num 0.0
cl sitting_down
num 0.0
cl standing_up
num 0.0
cl standing
num 0.0
cl walking
num 0.0
cl running
num 0.0
cl going_up
num 0.0
cl going_down
num 0.0
cl sitting
num 0.0
cl sitting_down
num 0.0
cl standing_up
num 0.0
cl standing
num 0.0
cl walking
num 0.0
num 50.0
cl running
num 0.0
num 50.0
cl going_up
num 0.0
num 50.0
cl going_down
num 0.0
num 50.0
cl sitting
num 0.0
num 50.0
cl sitting_down
num 0.0
nu

cl going_down
num 0.0
num 50.0
num 100.0
num 150.0
num 200.0
num 250.0
num 300.0
num 350.0
num 400.0
num 450.0
cl sitting
num 0.0
num 50.0
num 100.0
num 150.0
num 200.0
num 250.0
num 300.0
num 350.0
num 400.0
num 450.0
cl sitting_down
num 0.0
num 50.0
num 100.0
num 150.0
num 200.0
num 250.0
num 300.0
num 350.0
num 400.0
num 450.0
cl standing_up
num 0.0
num 50.0
num 100.0
num 150.0
num 200.0
num 250.0
num 300.0
num 350.0
num 400.0
num 450.0
cl standing
num 0.0
num 50.0
num 100.0
num 150.0
num 200.0
num 250.0
num 300.0
num 350.0
num 400.0
num 450.0
cl walking
num 0.0
num 50.0
num 100.0
num 150.0
num 200.0
num 250.0
num 300.0
num 350.0
num 400.0
num 450.0
num 500.0
num 550.0
num 600.0
num 650.0
num 700.0
num 750.0
num 800.0
num 850.0
num 900.0
num 950.0
cl running
num 0.0
num 50.0
num 100.0
num 150.0
num 200.0
num 250.0
num 300.0
num 350.0
num 400.0
num 450.0
num 500.0
num 550.0
num 600.0
num 650.0
num 700.0
num 750.0
num 800.0
num 850.0
num 900.0
num 950.0
cl going_up
num 0.0
num 50.0
nu

num 600.0
num 650.0
num 700.0
num 750.0
num 800.0
num 850.0
num 900.0
num 950.0
cl sitting_down
num 0.0
num 50.0
num 100.0
num 150.0
num 200.0
num 250.0
num 300.0
num 350.0
num 400.0
num 450.0
num 500.0
num 550.0
num 600.0
num 650.0
num 700.0
num 750.0
num 800.0
num 850.0
num 900.0
num 950.0
cl standing_up
num 0.0
num 50.0
num 100.0
num 150.0
num 200.0
num 250.0
num 300.0
num 350.0
num 400.0
num 450.0
num 500.0
num 550.0
num 600.0
num 650.0
num 700.0
num 750.0
num 800.0
num 850.0
num 900.0
num 950.0
cl standing
num 0.0
num 50.0
num 100.0
num 150.0
num 200.0
num 250.0
num 300.0
num 350.0
num 400.0
num 450.0
num 500.0
num 550.0
num 600.0
num 650.0
num 700.0
num 750.0
num 800.0
num 850.0
num 900.0
num 950.0
cl walking
num 0.0
num 50.0
num 100.0
num 150.0
num 200.0
num 250.0
num 300.0
num 350.0
num 400.0
num 450.0
num 500.0
num 550.0
num 600.0
num 650.0
num 700.0
num 750.0
num 800.0
num 850.0
num 900.0
num 950.0
num 1000.0
num 1050.0
num 1100.0
num 1150.0
num 1200.0
num 1250.0
num 1300.0
n

num 1100.0
num 1150.0
num 1200.0
num 1250.0
num 1300.0
num 1350.0
num 1400.0
num 1450.0
num 1500.0
num 1550.0
num 1600.0
num 1650.0
num 1700.0
num 1750.0
num 1800.0
num 1850.0
num 1900.0
num 1950.0
cl going_up
num 0.0
num 50.0
num 100.0
num 150.0
num 200.0
num 250.0
num 300.0
num 350.0
num 400.0
num 450.0
num 500.0
num 550.0
num 600.0
num 650.0
num 700.0
num 750.0
num 800.0
num 850.0
num 900.0
num 950.0
num 1000.0
num 1050.0
num 1100.0
num 1150.0
num 1200.0
num 1250.0
num 1300.0
num 1350.0
num 1400.0
num 1450.0
num 1500.0
num 1550.0
num 1600.0
num 1650.0
num 1700.0
num 1750.0
num 1800.0
num 1850.0
num 1900.0
num 1950.0
cl going_down
num 0.0
num 50.0
num 100.0
num 150.0
num 200.0
num 250.0
num 300.0
num 350.0
num 400.0
num 450.0
num 500.0
num 550.0
num 600.0
num 650.0
num 700.0
num 750.0
num 800.0
num 850.0
num 900.0
num 950.0
num 1000.0
num 1050.0
num 1100.0
num 1150.0
num 1200.0
num 1250.0
num 1300.0
num 1350.0
num 1400.0
num 1450.0
num 1500.0
num 1550.0
num 1600.0
num 1650.0
num 1700

num 1400.0
num 1450.0
num 1500.0
num 1550.0
num 1600.0
num 1650.0
num 1700.0
num 1750.0
num 1800.0
num 1850.0
num 1900.0
num 1950.0
cl sitting_down
num 0.0
num 50.0
num 100.0
num 150.0
num 200.0
num 250.0
num 300.0
num 350.0
num 400.0
num 450.0
num 500.0
num 550.0
num 600.0
num 650.0
num 700.0
num 750.0
num 800.0
num 850.0
num 900.0
num 950.0
num 1000.0
num 1050.0
num 1100.0
num 1150.0
num 1200.0
num 1250.0
num 1300.0
num 1350.0
num 1400.0
num 1450.0
num 1500.0
num 1550.0
num 1600.0
num 1650.0
num 1700.0
num 1750.0
num 1800.0
num 1850.0
num 1900.0
num 1950.0
cl standing_up
num 0.0
num 50.0
num 100.0
num 150.0
num 200.0
num 250.0
num 300.0
num 350.0
num 400.0
num 450.0
num 500.0
num 550.0
num 600.0
num 650.0
num 700.0
num 750.0
num 800.0
num 850.0
num 900.0
num 950.0
num 1000.0
num 1050.0
num 1100.0
num 1150.0
num 1200.0
num 1250.0
num 1300.0
num 1350.0
num 1400.0
num 1450.0
num 1500.0
num 1550.0
num 1600.0
num 1650.0
num 1700.0
num 1750.0
num 1800.0
num 1850.0
num 1900.0
num 1950.0
cl 

num 1550.0
num 1600.0
num 1650.0
num 1700.0
num 1750.0
num 1800.0
num 1850.0
num 1900.0
num 1950.0
num 2000.0
num 2050.0
num 2100.0
num 2150.0
num 2200.0
num 2250.0
num 2300.0
num 2350.0
num 2400.0
num 2450.0
num 2500.0
num 2550.0
num 2600.0
num 2650.0
num 2700.0
num 2750.0
num 2800.0
num 2850.0
num 2900.0
num 2950.0
cl going_down
num 0.0
num 50.0
num 100.0
num 150.0
num 200.0
num 250.0
num 300.0
num 350.0
num 400.0
num 450.0
num 500.0
num 550.0
num 600.0
num 650.0
num 700.0
num 750.0
num 800.0
num 850.0
num 900.0
num 950.0
num 1000.0
num 1050.0
num 1100.0
num 1150.0
num 1200.0
num 1250.0
num 1300.0
num 1350.0
num 1400.0
num 1450.0
num 1500.0
num 1550.0
num 1600.0
num 1650.0
num 1700.0
num 1750.0
num 1800.0
num 1850.0
num 1900.0
num 1950.0
num 2000.0
num 2050.0
num 2100.0
num 2150.0
num 2200.0
num 2250.0
num 2300.0
num 2350.0
num 2400.0
num 2450.0
num 2500.0
num 2550.0
num 2600.0
num 2650.0
num 2700.0
num 2750.0
num 2800.0
num 2850.0
num 2900.0
num 2950.0
cl sitting
num 0.0
num 50.0
nu

num 750.0
num 800.0
num 850.0
num 900.0
num 950.0
num 1000.0
num 1050.0
num 1100.0
num 1150.0
num 1200.0
num 1250.0
num 1300.0
num 1350.0
num 1400.0
num 1450.0
num 1500.0
num 1550.0
num 1600.0
num 1650.0
num 1700.0
num 1750.0
num 1800.0
num 1850.0
num 1900.0
num 1950.0
num 2000.0
num 2050.0
num 2100.0
num 2150.0
num 2200.0
num 2250.0
num 2300.0
num 2350.0
num 2400.0
num 2450.0
num 2500.0
num 2550.0
num 2600.0
num 2650.0
num 2700.0
num 2750.0
num 2800.0
num 2850.0
num 2900.0
num 2950.0
cl walking
num 0.0
num 50.0
num 100.0
num 150.0
num 200.0
num 250.0
num 300.0
num 350.0
num 400.0
num 450.0
num 500.0
num 550.0
num 600.0
num 650.0
num 700.0
num 750.0
num 800.0
num 850.0
num 900.0
num 950.0
num 1000.0
num 1050.0
num 1100.0
num 1150.0
num 1200.0
num 1250.0
num 1300.0
num 1350.0
num 1400.0
num 1450.0
num 1500.0
num 1550.0
num 1600.0
num 1650.0
num 1700.0
num 1750.0
num 1800.0
num 1850.0
num 1900.0
num 1950.0
num 2000.0
num 2050.0
num 2100.0
num 2150.0
num 2200.0
num 2250.0
num 2300.0
num 2

num 600.0
num 650.0
num 700.0
num 750.0
num 800.0
num 850.0
num 900.0
num 950.0
num 1000.0
num 1050.0
num 1100.0
num 1150.0
num 1200.0
num 1250.0
num 1300.0
num 1350.0
num 1400.0
num 1450.0
num 1500.0
num 1550.0
num 1600.0
num 1650.0
num 1700.0
num 1750.0
num 1800.0
num 1850.0
num 1900.0
num 1950.0
num 2000.0
num 2050.0
num 2100.0
num 2150.0
num 2200.0
num 2250.0
num 2300.0
num 2350.0
num 2400.0
num 2450.0
num 2500.0
num 2550.0
num 2600.0
num 2650.0
num 2700.0
num 2750.0
num 2800.0
num 2850.0
num 2900.0
num 2950.0
cl sitting_down
num 0.0
num 50.0
num 100.0
num 150.0
num 200.0
num 250.0
num 300.0
num 350.0
num 400.0
num 450.0
num 500.0
num 550.0
num 600.0
num 650.0
num 700.0
num 750.0
num 800.0
num 850.0
num 900.0
num 950.0
num 1000.0
num 1050.0
num 1100.0
num 1150.0
num 1200.0
num 1250.0
num 1300.0
num 1350.0
num 1400.0
num 1450.0
num 1500.0
num 1550.0
num 1600.0
num 1650.0
num 1700.0
num 1750.0
num 1800.0
num 1850.0
num 1900.0
num 1950.0
num 2000.0
num 2050.0
num 2100.0
num 2150.0
num

In [23]:
# save test labels 
for s in test_vids:
    #print('session ' + s)
    lab_path = path + '/labels-hand/padd_sessn_' + s + '_labels.csv'
    df = pd.read_csv(lab_path, index_col=0)
    
    lab_path = path + '/labels-hand/' + 'sess_' + s + '_labels.csv'
    df.columns = ['background', 'walking', 'running', 'going_up', 'going_down', 'sitting',
                     'sitting_down', 'standing_up', 'standing']
    df.to_csv(lab_path)
    

In [69]:
#print(new_labels)
for lab in new_labels:
    print(lab, lab.shape)
    unique, counts = np.unique(lab, return_counts=True)

    print(np.asarray((unique, counts)).T)

[0 0 0 ... 0 0 0] (86937,)
[[    0 81737]
 [    1   600]
 [    2   600]
 [    3   450]
 [    4   400]
 [    5   450]
 [    6   500]
 [    7   750]
 [    8   550]
 [    9   600]
 [   10   300]]
[0 0 0 ... 0 0 0] (81377,)
[[    0 76477]
 [    1   450]
 [    2   450]
 [    3   650]
 [    4   400]
 [    5   500]
 [    6   700]
 [    7   250]
 [    8   400]
 [    9   300]
 [   10   800]]
[0 0 0 ... 0 0 0] (57120,)
[[    0 51670]
 [    1   350]
 [    2   700]
 [    3   450]
 [    4   650]
 [    5   400]
 [    6   750]
 [    7   500]
 [    8   550]
 [    9   550]
 [   10   550]]
[0 0 0 ... 0 0 0] (57123,)
[[    0 52323]
 [    1   650]
 [    2   250]
 [    3   550]
 [    4   550]
 [    5   550]
 [    6   300]
 [    7   450]
 [    8   550]
 [    9   600]
 [   10   350]]
[0 0 0 ... 0 0 0] (73899,)
[[    0 69249]
 [    1   450]
 [    2   500]
 [    3   400]
 [    4   500]
 [    5   600]
 [    6   250]
 [    7   550]
 [    8   450]
 [    9   450]
 [   10   500]]
